__3. Create a SQLite database called `CourseData.db` in this folder. The database should exactly match your ERD. Populate the database with data from the CSV files.__

- CSV files are best imported using the SQLite3 tool we discussed in class. Experiment with it to get the imports right before writing any SQL.
- You will likely have to import the CSV file data into tables that you will ultimately drop when the database is completed. To keep track of them all, please the prefix the table name with `import_` to indicate that the table contains raw source data.
- Use SQL to create and populate the tables in your ERD. The code will likely look a lot like what we did in class, with lots of JOINs. You should implement FOREIGN KEY constraints (With cascading updates/deletes) as well.
- Capture all of your SQL DDL and DML code in a new notebook called `CourseDataETL.ipynb` in this project folder. Use Markdown to annotate your work as you go along. Also, make sure you can re-run your code from scratch to rebuild the database when needed.

__Import Packages__

In [1]:
%load_ext sql
import pandas as pd
import sqlite3

In [2]:
%sql sqlite:///CourseData.db
conn = sqlite3.connect('CourseData.db')

__Create Tables__

In [3]:
%%sql
DROP TABLE IF EXISTS Class;
CREATE TABLE Class (
    ClassID INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    CatalogCourseID INTEGER NOT NULL,
    ProfessorID INTEGER NOT NULL,
    Term TEXT NOT NULL,
    CRN INTEGER NOT NULL,
    Section TEXT NOT NULL,
    Credits INTEGER NOT NULL,
    Title TEXT NOT NULL,
    Timecodes TEXT NOT NULL,
    Meetings TEXT NOT NULL,
    Capacity INTEGER NOT NULL,
    Actual INTEGER NOT NULL,
    Remaining INTEGER NOT NULL
);

DROP TABLE IF EXISTS CatalogCourse;
CREATE TABLE CatalogCourse (
    CatalogCourseID INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    CatalogYear TEXT NOT NULL,
    ProgramID INTEGER NOT NULL,
    Title TEXT NOT NULL,
    Credits REAL,
    Prereqs TEXT,
    Coreqs TEXT,
    Fees TEXT,
    Attributes TEXT,
    Description TEXT
);

DROP TABLE IF EXISTS Meeting;
CREATE TABLE Meeting (
    MeetingID INTEGER PRIMARY KEY,
    ClassID INTEGER NOT NULL,
    LocationID INTEGER NOT NULL,
    Day TEXT NOT NULL,
    StartTime TEXT NOT NULL,
    EndTime TEXT NOT NULL
);

DROP TABLE IF EXISTS Program;
CREATE TABLE Program (
    ProgramID INTEGER PRIMARY KEY,
    ProgramName TEXT NOT NULL,
    ProgramCode TEXT NOT NULL
);

DROP TABLE IF EXISTS Location;
CREATE TABLE Location (
    LocationID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

DROP TABLE IF EXISTS Professor;
CREATE TABLE Professor (
    ProfessorID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Extract from CSV

In [4]:
# Class & Meeting CSVs

terms = ['Fall2014','Fall2015','Fall2016','Fall2017','Fall2018',
         'Spring2015','Spring2016','Spring2017','Spring2018','Spring2019',
         'SpringBreak2017',
         'Summer2015','Summer2016','Summer2017','Summer2018',
         'Winter2015','Winter2016','Winter2017','Winter2018']

for term in terms:
    filepath = 'SourceData/'+term+'/courses.csv'
    data = pd.read_csv(filepath)
    data.to_sql('import_courses',conn,if_exists='append',index=False)
    
    filepath = 'SourceData/'+term+'/course_meetings.csv'
    data = pd.read_csv(filepath)
    data.to_sql('import_course_meetings',conn,if_exists='append',index=False)

In [5]:
%%sql
SELECT 
    (SELECT Count(*) FROM import_courses) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM import_courses)) as 'DistinctCount'

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
15937,15937


In [6]:
%%sql
SELECT 
    (SELECT Count(*) FROM import_course_meetings) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM import_course_meetings)) as 'DistinctCount'

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
284907,284847


In [7]:
# Catalog CSV
catalog_years = ['2017_2018', '2018_2019']

for cat_year in catalog_years:
    filepath = 'SourceData/Catalogs/CourseCatalog'+cat_year+'.csv'
    data = pd.read_csv(filepath)
    data['cat_year'] = cat_year
    data.to_sql('import_catalog',conn,if_exists='append',index=False)

In [8]:
%%sql
SELECT 
    (SELECT Count(*) FROM import_catalog) as 'RawCount',
    (SELECT Count(*) FROM (SELECT DISTINCT * FROM import_catalog)) as 'DistinctCount';

 * sqlite:///CourseData.db
Done.


RawCount,DistinctCount
4440,4440


## Import Data into ERD Tables

- Ordered by entity strength. First do the tables with no foreign keys. When creating a table, be sure the tables foreign keys have already been created first.

In [9]:
%%sql

DELETE FROM Professor;

INSERT INTO Professor (Name)
SELECT DISTINCT primary_instructor
FROM import_courses
WHERE primary_instructor != 'TBA' AND primary_instructor NOT LIKE '%/%';

 * sqlite:///CourseData.db
0 rows affected.
1095 rows affected.


[]

In [10]:
%%sql

DELETE FROM Location;

INSERT INTO Location (Location)
SELECT DISTINCT location
FROM import_course_meetings

 * sqlite:///CourseData.db
0 rows affected.
207 rows affected.


[]

In [11]:
%%sql

DELETE FROM Program;

INSERT INTO Program (ProgramName, ProgramCode)
SELECT DISTINCT program_name, program_code
FROM import_catalog;

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

In [12]:
%%sql

DELETE FROM CatalogCourse;

INSERT INTO CatalogCourse (CatalogID, CatalogYear, ProgramID, Title, Credits, Prereqs, Coreqs, Fees, Attributes, Description)
SELECT DISTINCT catalog_id, cat_year, ProgramID, course_title, credits, prereqs, coreqs, fees, attributes, description
FROM import_catalog
    JOIN Program ON (program_code = ProgramCode)

 * sqlite:///CourseData.db
0 rows affected.
4440 rows affected.


[]

- CatalogYear column was made in the for loop while importing the Catalog CSVs: cat_year

In [18]:
%%sql

DELETE FROM Class;

INSERT INTO Class (CatalogID, CatalogCourseID, ProfessorID, Term, CRN, Section, Credits, Title, Timecodes, Meetings, Capacity, Actual, Remaining)
SELECT DISTINCT CatalogID, CatalogCourseID, ProfessorID, term, crn, section, import_courses.credits, title, timecodes, meetings, cap, act, rem
FROM import_courses
    LEFT JOIN Professor ON (primary_instructor = Professor.Name)
    LEFT JOIN CatalogCourse ON (catalog_id = CatalogID);

 * sqlite:///CourseData.db
0 rows affected.
(sqlite3.OperationalError) ambiguous column name: title
[SQL: INSERT INTO Class (CatalogID, CatalogCourseID, ProfessorID, Term, CRN, Section, Credits, Title, Timecodes, Meetings, Capacity, Actual, Remaining)
SELECT DISTINCT CatalogID, CatalogCourseID, ProfessorID, term, crn, section, import_courses.credits, title, timecodes, meetings, cap, act, rem
FROM import_courses
    LEFT JOIN Professor ON (primary_instructor = Professor.Name)
    LEFT JOIN CatalogCourse ON (catalog_id = CatalogID);]
(Background on this error at: http://sqlalche.me/e/e3q8)
